In [4]:
import pandas as pd
import numpy as np
import os

#读取18年到19年2月的订单量数据表，提取字段整合成一张订单量数据表
first = 0
for i in os.listdir('/Users/xiujiayang/PycharmProjects/cardinalintern/data_process_train/order_data'):
    if first == 0:
        order_data = pd.read_csv('/Users/xiujiayang/PycharmProjects/cardinalintern/data_process_train/order_data/'+i,
                             usecols=['Calendar Day','Banner Name','Sold-to party','Sold-to party Name','Plant Code',
                              'Plant Name','Material','Material Description','Final Expect Qty'])
        first = 1
    else:
        new_data = pd.read_csv('/Users/xiujiayang/PycharmProjects/cardinalintern/data_process_train/order_data/'+i,
                             usecols=['Calendar Day','Banner Name','Sold-to party','Sold-to party Name','Plant Code',
                              'Plant Name','Material','Material Description','Final Expect Qty'])
        order_data = order_data.append(new_data)
print('订单量数据读取完毕，订单量数据维度为',order_data.shape)

#字段名称重新命名，命名格式为小写字母，如果多个单词使用下划线隔开，例如date、order_qty等
order_data.rename(columns={'Calendar Day':'date','Banner Name':'customer_group','Sold-to party':'customer_code',
                          'Sold-to party Name':'customer_name','Plant Code':'plant_code','Plant Name':'plant_name',
                          'Material':'material','Material Description':'material_description','Final Expect Qty':'oder_qty'},
                           inplace=True)
print('完成字段重新命名')


In [ ]:
#读取商品数据，提取1.2中的字段整合成一张商品信息表
sku_data = pd.read_excel('/Users/xiujiayang/PycharmProjects/cardinalintern/data_process_train/sku_data/Material+PH+report+GCC+-+Mar%2c2019.xlsx')
sku_data = sku_data.loc[:,['Material Code','L5-Brand','L3-Category','PC EAN']]
sku_data.rename(columns={'Material Code':'material','PC EAN':'bar_code','L5-Brand':'brand','L3-Category':'category'},
                inplace=True)
print('商品数据读取完毕，商品数据维度为',sku_data.shape)
print(sku_data.iloc[:5])


商品数据读取完毕，商品数据维度为 (18852, 4)
   material   brand   category      bar_code
0  15092579  MAGNUM  ICE CREAM  6.909493e+12
1  15092580  MAGNUM  ICE CREAM  6.909493e+12
2  15092581   KNORR    SAVOURY  4.902112e+12
3  15092582   KNORR    SAVOURY  9.902089e+12
4  15092583   KNORR    SAVOURY  6.922849e+12


In [3]:
#订单量数据表中，根据商品编码从商品信息表中匹配上品牌，品类和bar_code
data = pd.merge(order_data,sku_data,on='material')
print('数据合并完毕，维度为',data.shape)
data1 = data.loc[:,['customer_group','customer_name','plant_name','bar_code','date']]
data1 = data.groupby(['customer_group','customer_name','plant_name','bar_code','date']).oder_qty.sum()
data1.iloc[:5]

数据合并完毕，维度为 (1544633, 12)


customer_group  customer_name   plant_name               bar_code      date    
CARREFOUR       上海家源商业有限公司天山路店  IC-CD-Shanghai South DC  6.909493e+12  20180706    0
                                                                       20180710    5
                                                                       20180829    2
                                                                       20180831    2
                                                                       20180903    0
Name: oder_qty, dtype: int64

In [ ]:
#处理bar_code与品类品牌一一对应关系
dict_bar2brand1, bar_code_list = {},[]
for i in range(data.shape[0]):
    dict_bar2brand1[(data.iloc[i]['bar_code'],data.iloc[i]['brand'],
                     data.iloc[i]['category'])] = dict_bar2brand1.get((data.iloc[i]['bar_code'],
                                                                       data.iloc[i]['brand'],data.iloc[i]['category']),1)+1
    if data.iloc[i]['bar_code'] not in bar_code_list:
        bar_code_list.append(data.iloc[i]['bar_code'])
    if i % (data.shape[0]//10) == 0:
        print('完成1/10')
dict_bar2brand = {}
for i in bar_code_list:
    max_count = 0
    for j in dict_bar2brand1.keys():
        if i == j[0]:
            if dict_bar2brand1[j] > max_count:
                dict_bar2brand[i] = (j[1],j[2])
count = 0
for i in range(data.shape[0]):
    if (data.iloc[i]['brand'], data.iloc[i]['category']) != dict_bar2brand[data.iloc[i]['bar_code']]:
        data.iloc[i]['brand'],data.iloc[i]['category'] = dict_bar2brand[data.iloc[i]['bar_code']][0],dict_bar2brand[data.iloc[i]['bar_code']][1]
        count += 1
print('完成bar_code与品类品牌一一对应关系的处理,改进',count,'个错误对应')
        


完成1/10
完成1/10
完成1/10
完成1/10
完成1/10
完成1/10
完成1/10
完成1/10
完成1/10
完成1/10
完成1/10


/Users/xiujiayang/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
#追溯历史上订单量数据
#2019年客户+bar_code的仓库架构

ic_cold_count = 0 
rows = [i for i in range(data.shape[0]) if 'IC' in str(data.iloc[i]['plant_name']) or 'Cold' in str(data.iloc[i]['plant_name'])]
print('去掉仓库里带IC、Cold的数据,一共',len(rows),'个数据')
data.drop(rows,inplace=True)
data_2019 = data.loc[data['date'].isin (['2019010'+str(i) for i in range(1,10)]+
                     ['201901'+str(i) for i in range(10,32)]+['2019020'+str(i) for i in range(1,10)]+
                     ['201902'+str(i) for i in range(10,29)])]
data_not_2019 = data.loc[~data['date'].isin (['2019010'+str(i) for i in range(1,10)]+
                     ['201901'+str(i) for i in range(10,32)]+['2019020'+str(i) for i in range(1,10)]+
                     ['201902'+str(i) for i in range(10,29)])]
#data_2019 = data_2019.groupby(['customer_name','bar_code'])
dict_bar_customer = {}
bar_cur_list = []
for i in range(data_2019.shape[0]):
    dict_bar_customer[(data_2019.iloc[i]['bar_code'],data_2019.iloc[i]['customer_name'],
                      data_2019.iloc[i]['plant_name'])] = dict_bar_customer.get((data_2019.iloc[i]['bar_code'],
                                                        data_2019.iloc[i]['customer_name'],data_2019.iloc[i]['plant_name']),1) + 1
    if (data_2019.iloc[i]['bar_code'],data_2019.iloc[i]['customer_name']) not in bar_cur_list:
        bar_cur_list.append((data_2019.iloc[i]['bar_code'],data_2019.iloc[i]['customer_name']))
    if i % (data_2019.shape[0]//5) == 0:
        print('完成1/5')
dict_bar_customer_plant = {}
for i in bar_cur_list:
    max_count = 0
    for j in dict_bar_customer.keys():
        if i == (j[0],j[1]):
            if dict_bar_customer[j] > max_count:
                dict_bar_customer_plant[i] = j[2]

#一部分客户+bar_code不在19年最新的仓库架构里
count = 0
remove_list = []
for i in range(data_not_2019.shape[0]):
    if (data_not_2019.iloc[i]['bar_code'],data_not_2019.iloc[i]['customer_name']) not in dict_bar_customer_plant.keys():
        count += 1
        remove_list.append(i)
    if i % (data_not_2019.shape[0]//10) == 0:
        print('完成1/10')   
print('得舍弃订货量',count)
data_not_2019.drop(remove_list,inplace=True)
update_count = 0
for i in range(data_not_2019.shape[0]):
    if  data_not_2019.iloc[i]['plant_name'] != dict_bar_customer_plant[(data_not_2019.iloc[i]['bar_code'],
                                                                       data_not_2019.iloc[i]['customer_name'])]:
        data_not_2019.iloc[i]['plant_name'] = dict_bar_customer_plant[(data_not_2019.iloc[i]['bar_code'],
                                                                       data_not_2019.iloc[i]['customer_name'])]
        update_count += 1
print('追溯历史上订单量数据完成，改进', update_count,'个数据')
            
                     
                     
                     

In [ ]:
#将订单量数据按照客户组+客户+追溯后的仓库+bar_code+日期汇总【表A】
data_a = pd.concat([data_2019,data_not_2019])
data_table_a = data_a.copy()
data_table_a = data_table_a.groupby(['customer_group','customer_name','plant_name','bar_code','date']).oder_qty.sum()
data_table_a.to_csv('table_a.csv')
data_table_a.head

In [ ]:
#增加一列月份，格式为“2019-01-01”(即使用每月1号来表示当月)
data_a['month'] = [str(x)[:4] +'-'+ str(x)[4:6]+'-'+'01' for x in data_a['date']]

In [ ]:
#将订单量数据按照客户组+追溯后的仓库+bar_code+月份汇总【表B】
data_b = data_a.loc[:,['customer_group','plant_name','bar_code','month','oder_qty']]
data_table_b = data_b.groupby(['customer_group','plant_name','bar_code','month']).oder_qty.sum()
data_table_b.to_csv('table_b.csv')
data_table_b.head

In [ ]:
import datetime

data_a['date'] = pd.to_datetime(data_a['date'],format='%Y%m%d')
min_date = np.min(data_a['date'])
max_date = np.max(data_a['date'])
data_c = data_a.groupby(['bar_code','date']).oder_qty.sum()
data_c = data_c.unstack()

def getEveryDay(begin_date,end_date):
    date_list = []
    while begin_date <= end_date:
        date_str = begin_date.strftime("%Y-%m-%d")
        date_list.append(date_str)
        begin_date += datetime.timedelta(days=1)
    return date_list
datelist = getEveryDay(min_date,max_date)
data_table_c = pd.DataFrame(columns=datelist,index=data_c.index)
for i in data_c.index:
    for j in datelist:
        if j in data_c.columns:
            data_table_c.loc[i,j] = data_c.loc[i,j]
        else:
            data_table_c.loc[i,j] = 0
data_table_c.to_csv('table_c.vsv')
data_table_c.head 

In [ ]:
import matplotlib.pyplot as plt
total = data_table_c.shape[0]
print(total)
for i in range(data_table_c.shape[0]):
    plt.figure(i)
    data_table_c.iloc[i].plot()